# PCA for Data Visualization

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler,OrdinalEncoder
from sklearn.decomposition import PCA
from sklearn import set_config
set_config(transform_output='pandas')
pd.set_option('display.max_columns',100)




In [ ]:
df = pd.read_csv("Data/modified-Data_Cortex_Nuclear.csv")
# Dropping unique IDs
df = df.drop(columns=['MouseID'])
df.info()
df.head()



In [ ]:
possible_targets = ['Type of Mouse',"Treatment",'Training','class',]
for col in possible_targets:
    print(f'\n- {col}:')
    print(df[col].value_counts())



In [ ]:
# Define X and y
target = "Training"
X = df.drop(columns = possible_targets)
y = df[target]


In [ ]:
import missingno as msno
msno.matrix(X)


In [ ]:
# Fill in nulls with 0 and verify all nulls are addressed
X = X.fillna(0)
X.isna().sum().sum()



In [ ]:
# Instantiate Standard Scaler
scaler = StandardScaler()
# Fit & transform data.
scaled_df = scaler.fit_transform(X)
scaled_df.head()


In [ ]:
# Calculating the correlation matrix and plotting
corr = scaled_df.corr()
sns.heatmap(corr, cmap='coolwarm');



In [ ]:
# Creating a scatter_matrix with pandas
pd.plotting.scatter_matrix(scaled_df, figsize=(40,40));



In [ ]:
# randomly select features to plot
np.random.seed(42)
random_features = np.random.choice(scaled_df.columns,3)
# plot thee randomly selected features
sns.pairplot(scaled_df,  vars=random_features);



In [ ]:
# Concatenate scaled features with target
plot_df = pd.concat([scaled_df, df[target]], axis=1)
# Plot with color coding based on target
g = sns.pairplot(data=plot_df,  vars=random_features, hue='Training')
g.fig.suptitle('Visualizing Raw Features - Colored by Training', y=1.01);



In [ ]:
# Instantiate PCA to make 3 principal components
pca = PCA(n_components=3)
# Create and define the principal components
principal_components = pca.fit_transform(scaled_df)
# Preview the results
principal_components.head()



In [ ]:
# Variance explained by each PC
pca.explained_variance_ratio_



In [ ]:
# Sum of variance explained by 3 principal components
pca.explained_variance_ratio_.sum()



In [ ]:
# Concatenate principal components with target
plot_df_pca = pd.concat([principal_components, df[target]], axis=1)
# Plot with color coding based on target
g_pca = sns.pairplot(data=plot_df_pca,  vars=principal_components.columns, hue='Training')
g_pca.fig.suptitle('Visualizing First 3 PCs - Colored by Training', y=1.01);



In [ ]:
import plotly.express as px
import plotly.io as pio
# Make a 3d scatter plot with a PC on each axis and color by the target
fig = px.scatter_3d(plot_df_pca, x='pca0',y='pca1',z='pca2', width=800, height=600, color = "Training")
fig



In [ ]:
def update_scatter3d(fig):
    fig.update_traces({'marker':{'size':3}})
    fig.show(config={'scrollZoom':False})
update_scatter3d(fig)



In [ ]:
# See avaialbe templates
pio.templates



In [ ]:
# Make a 3d scatter plot with a PC on each axis and color by the target
# Change template style to plotly_dark
fig = px.scatter_3d(plot_df_pca, x='pca0',y='pca1',z='pca2', width=800, height=600, color = "Training", template = 'plotly_dark')
update_scatter3d(fig)


# PCA to Speed up Machine Learning Algorithms

## PCA for Supervised Machine Learning

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

pd.set_option('display.max_columns',200)
pd.set_option("display.max_info_rows", 800)
pd.set_option('display.max_info_columns',800)

from sklearn import set_config
set_config(transform_output='pandas')


In [ ]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def classification_metrics(y_true, y_pred, label='',
                           output_dict=False, figsize=(8,4),
                           normalize='true', cmap='Blues',
                           colorbar=False):
    # Get the classification report
    report = classification_report(y_true, y_pred)
    ## Print header and report
    header = "-"*70
    print(header, f" Classification Metrics: {label}", header, sep='\n')
    print(report)
    ## CONFUSION MATRICES SUBPLOTS
    fig, axes = plt.subplots(ncols=2, figsize=figsize)
    # create a confusion matrix  of raw counts
    ConfusionMatrixDisplay.from_predictions(y_true, y_pred,
                normalize=None, cmap='gist_gray', colorbar=colorbar,
                ax = axes[0],);
    axes[0].set_title("Raw Counts")
    # create a confusion matrix with the test data
    ConfusionMatrixDisplay.from_predictions(y_true, y_pred,
                normalize=normalize, cmap=cmap, colorbar=colorbar,
                ax = axes[1]);
    axes[1].set_title("Normalized Confusion Matrix")
    # Adjust layout and show figure
    fig.tight_layout()
    plt.show()
    # Return dictionary of classification_report
    if output_dict==True:
        report_dict = classification_report(y_true, y_pred, output_dict=True)
        return report_dict


def evaluate_classification(model, X_train, y_train, X_test, y_test,
                         figsize=(6,4), normalize='true', output_dict = False,
                            cmap_train='Blues', cmap_test="Reds",colorbar=False):
    # Get predictions for training data
    y_train_pred = model.predict(X_train)
    # Call the helper function to obtain regression metrics for training data
    results_train = classification_metrics(y_train, y_train_pred, #verbose = verbose,
                                     output_dict=True, figsize=figsize,
                                         colorbar=colorbar, cmap=cmap_train,
                                     label='Training Data')
    print()
    # Get predictions for test data
    y_test_pred = model.predict(X_test)
    # Call the helper function to obtain regression metrics for test data
    results_test = classification_metrics(y_test, y_test_pred, #verbose = verbose,
                                  output_dict=True,figsize=figsize,
                                         colorbar=colorbar, cmap=cmap_test,
                                    label='Test Data' )
    if output_dict == True:
        # Store results in a dataframe if ouput_frame is True
        results_dict = {'train':results_train,
                    'test': results_test}
        return results_dict



In [ ]:
# Get Data
df = pd.read_csv('Data/pd_speech_features.csv')
df.head()


In [ ]:
df.dtypes.value_counts()


In [ ]:
df.isna().sum().sum()


In [ ]:
# Define target and cols to drop
target_col = 'class'
drop_cols = ['id']
# Define X and y
y = df[target_col].copy()
X = df.drop(columns=[target_col,*drop_cols]).copy()



In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y, random_state=321)
X_train.head()



In [ ]:
# Instantiate Standard Scaler
scaler = StandardScaler()
# Fit & transform data.
X_train_tf = scaler.fit_transform(X_train)
X_test_tf = scaler.transform(X_test)



In [ ]:
# Check for class balance of target
y_train.value_counts(1)



In [ ]:
from imblearn.over_sampling import SMOTE,SMOTENC
smote = SMOTE()
X_train_sm, y_train_sm = smote.fit_resample(X_train_tf, y_train)
y_train_sm.value_counts()



In [ ]:
# Import datetime library
import datetime as dt

# Record the start time
start = dt.datetime.now()

clf = RandomForestClassifier(random_state=42)

clf.fit(X_train_sm, y_train_sm)

# Record the end time and calc duration
end = dt.datetime.now()
dur_baseline = end-start

evaluate_classification(clf, X_train_sm,y_train_sm, X_test_tf, y_test)
print(f'Training time was: {dur_baseline}')



### PCA

In [ ]:
# Instantiate & fit data using PCA
pca = PCA(n_components=3)
X_train_pca = pca.fit_transform(X_train_sm)
X_test_pca = pca.transform(X_test_tf)
X_train_pca.head()



In [ ]:
# Record the start time
start = dt.datetime.now()

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_pca, y_train_sm)

# Record the end time and calc duration
end = dt.datetime.now()
dur_pca = end-start

evaluate_classification(clf, X_train_pca,y_train_sm, X_test_pca, y_test)
print(f'Training time was: {dur_pca}')


In [ ]:
# Compare speeds before and after PCA
compare_speed = dur_baseline/dur_pca
print(f"Using PCs was {compare_speed:.2f} times faster!")



In [ ]:
# Initialize and fit PCA
pca = PCA()
pca.fit(X_train_sm, y_train)



In [ ]:
# Determine how much variance is explained by each PC
explained = pd.Series(pca.explained_variance_ratio_, name='Explained Variance Ratio')
explained



In [ ]:
# plot the cumulative sum of the percentage of explained variance for each component and those before it.
ax = explained.cumsum().plot(marker='.')
# add a line to mark .9 (or 90%) variance explained
ax.axhline(.9, color='k');



### Specifying the Explained Variance
Rather than specifying the number of components to return, an alternate method is to specify the minimum proportion of explained variance you are willing to accept. PCA will automatically reduce the number of components just enough to meet your specification.

To specify the proportion of variance, give the n_components argument a float between 0 and 1, and it will return the number of components required to explain the given variance.

The code below is an example of how to ensure that enough components are returned to explain 85% of the variance. Instead of using an integer in the n_components argument to designate the number of components, we use a decimal value to indicate the amount of variance to be explained. PCA() will then automatically use enough principal components to meet this level of explanation.

In [ ]:
# Define PCA to address 85% of the variance
pca85 = PCA(n_components=.85)
# fit and transform on training data
X_train_pca85 = pca85.fit_transform(X_train_sm)
# transform test data
X_test_pca85 = pca85.transform(X_test_tf)
# obtain the number of PCs used
pca85.n_components_



In [ ]:
# Record the start time
start = dt.datetime.now()

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_pca85, y_train_sm)

# Record the end time and calc duration
end = dt.datetime.now()
dur_pca_85 = end-start

evaluate_classification(clf, X_train_pca85,y_train_sm, X_test_pca85, y_test)
print(f'Training time was: {dur_pca_85}')



In [ ]:
compare_speed = dur_baseline/dur_pca_85
print(f"Using PC's with .85 was {compare_speed:.2f} times faster!")



# Feature Engineering: Overloaded Operators

In [ ]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vReZBM5OC6GLYbacisp_ToNiu3CLWxqPXw7mWBsdRjnYOFLWNufdQ4qd8u5qTzUF2_sBUAMEi5cgy1U/pub?gid=1040198428&single=true&output=csv')
df.head()


In [ ]:
#Summing Features:
df['TotalFamily'] = df['SibSp'] + df['Parch'] 
df = df.drop(['SibSp', 'Parch'], axis=1)
df.head()


In [ ]:
# Concatenating Features:

df['Age'] = df['Age'].round(-1)
df.head()


In [ ]:
df['GenderAge'] = df['Sex'] + df['Age'].astype('string')
df.drop(columns=['Sex','Age'], inplace=True)
df.head()


In [ ]:
# Squaring and Multiplying Features

df['NormedFare'] = df['Fare'] * df['Pclass']**2
df.drop(columns='Fare', inplace=True)
df.head()


# Feature Engineering: Strings

In [ ]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vReZBM5OC6GLYbacisp_ToNiu3CLWxqPXw7mWBsdRjnYOFLWNufdQ4qd8u5qTzUF2_sBUAMEi5cgy1U/pub?gid=1040198428&single=true&output=csv')
df.head()


In [ ]:
# Separating Features: Strings

# create 2 new columns, FirstName and LastName by splitting the Name column
df[['LastName','FirstName']] = df['Name'].str.split(',', expand=True)
# drop the 'Name' column
df.drop('Name', axis=1, inplace=True)
df.head()


In [ ]:
# Clean-up

df.loc[0,'FirstName']


In [ ]:
df['FirstName'] = df['FirstName'].str.strip()
df.loc[0, 'FirstName']


In [ ]:
# Combining Strings

df['Name'] = df['FirstName'] + ' ' + df['LastName']
df.drop(columns=['LastName','FirstName'], inplace= True)
df.head()


# Feature Engineering: Datetime

In [ ]:
import pandas as pd
df2 = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSrgrUnz8mdosU-_k0aECouymqwds_mlaHpYlXzRtf7MBJ4N1r1inCfSDebaXwTVfLtH133EhwKf3mi/pub?gid=394699239&single=true&output=csv',                  usecols=['date','price','bedrooms','bathrooms'])
df2.head()



In [ ]:
df2['date'] = pd.to_datetime(df2['date'])
df2.info()



In [ ]:
df2['year'] = df2['date'].dt.year
df2['month (numeric)'] = df2['date'].dt.month
df2['month (name)'] = df2['date'].dt.month_name()
df2['day of month'] = df2['date'].dt.day
df2['day of week (numeric)'] = df2['date'].dt.weekday
df2['day of week (name)'] = df2['date'].dt.day_name()
df2.head()



# Feature Engineering: Functions

In [ ]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vReZBM5OC6GLYbacisp_ToNiu3CLWxqPXw7mWBsdRjnYOFLWNufdQ4qd8u5qTzUF2_sBUAMEi5cgy1U/pub?gid=1040198428&single=true&output=csv')
df.head()


In [ ]:
# find the median fare price
median_fare = df['Fare'].median()
# define a function that returns 'Expensive' or 'Cheap'
def bin_fare(fare):
    if fare > median_fare:    
        return 'Expensive'  
    else:    
        return 'Cheap'



In [ ]:
# apply bin_fare() function to each item in the 'Fare' column
df['Fare'] = df['Fare'].apply(bin_fare)
df.head()



In [ ]:
df['Age'] = df['Age'].apply(lambda x: 'elderly' if x > 30 else 'young')
df.head()


# Feature Engineering 1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Preprocessing tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Setting options
n=800
pd.set_option('display.max_columns',n)
pd.set_option("display.max_info_rows", n)
pd.set_option('display.max_info_columns',n)
from sklearn import set_config
set_config(transform_output='pandas')



In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
def classification_metrics(y_true, y_pred, label="",
                           output_dict=False, figsize=(8,4),
                           normalize='true', cmap='Blues',
                           colorbar=False):
  # Get the classification report
  report = classification_report(y_true, y_pred)
  ## Print header and report
  header = "-"*70
  print(header, f" Classification Metrics: {label}", header, sep='\n')
  print(report)
  ## CONFUSION MATRICES SUBPLOTS
  fig, axes = plt.subplots(ncols=2, figsize=figsize)
  # create a confusion matrix  of raw counts
  ConfusionMatrixDisplay.from_predictions(y_true, y_pred,
                normalize=None, cmap='gist_gray', colorbar=colorbar,
                ax = axes[0],);
  axes[0].set_title("Raw Counts")
  # create a confusion matrix with the test data
  ConfusionMatrixDisplay.from_predictions(y_true, y_pred,
                normalize=normalize, cmap=cmap, colorbar=colorbar,
                ax = axes[1]);
  axes[1].set_title("Normalized Confusion Matrix")
  # Adjust layout and show figure
  fig.tight_layout()
  plt.show()
  # Return dictionary of classification_report
  if output_dict==True:
    report_dict = classification_report(y_true, y_pred, output_dict=True)
    return report_dict
    
    
    
def evaluate_classification(model, X_train, y_train, X_test, y_test,
                         figsize=(6,4), normalize='true', output_dict = False,
                            cmap_train='Blues', cmap_test="Reds",colorbar=False):
  # Get predictions for training data
  y_train_pred = model.predict(X_train)
  # Call the helper function to obtain regression metrics for training data
  results_train = classification_metrics(y_train, y_train_pred, #verbose = verbose,
                                     output_dict=True, figsize=figsize,
                                         colorbar=colorbar, cmap=cmap_train,
                                     label='Training Data')
  print()
  # Get predictions for test data
  y_test_pred = model.predict(X_test)
  # Call the helper function to obtain regression metrics for test data
  results_test = classification_metrics(y_test, y_test_pred, #verbose = verbose,
                                  output_dict=True,figsize=figsize,
                                         colorbar=colorbar, cmap=cmap_test,
                                    label='Test Data' )
  if output_dict == True:
    # Store results in a dataframe if ouput_frame is True
    results_dict = {'train':results_train,
                    'test': results_test}
    return results_dict



In [ ]:
# Load Data
df = pd.read_csv('pd_speech_features.csv')
df.head()



In [ ]:
df.info()



In [ ]:
# Check for null values
df.isna().sum().sum()



In [ ]:
# Checking the unique dtypes
df.dtypes.unique()



In [ ]:
# Define target
target_col = 'class'

# Make list of columns to drop. In this case just the id column for now
drop_cols = ['id']

# Define X and y
y = df[target_col].copy()
X = df.drop(columns=[target_col,*drop_cols]).copy()

# Train test split
X_train,X_test,y_train,y_test = train_test_split(X,y, random_state=321)



In [ ]:
# Scale
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)



In [ ]:
# Check the balance of the target
y.value_counts(1)



In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 42)
X_train_sm,y_train_sm = smote.fit_resample(X_train_scaled, y_train)
y_train_sm.value_counts()


In [ ]:
# Instantiate a default random forest with a random state for reproductibility
rf_clf = RandomForestClassifier(random_state=42)

# Fit on the training data
rf_clf.fit(X_train_sm, y_train_sm)

# Evaluate with the custom function
evaluate_classification(rf_clf, X_train_sm,y_train_sm, X_test_scaled, y_test)



In [ ]:
# Make a correlation heatmap of all features
corr = X_train_sm.corr()
plt.figure(figsize=(15,15))
sns.heatmap(corr,cmap='coolwarm');



In [ ]:
from collinearity import SelectNonCollinear
from sklearn.feature_selection import f_classif



In [ ]:
# Instantiate the selector, indicate theshold for r, also include the appropriate scoring metric for the task
selector = SelectNonCollinear(correlation_threshold = 0.75, scoring = f_classif)



In [ ]:
# Fit on the training data, use .values
selector.fit(X_train_sm.values, y_train_sm.values)



In [ ]:
# Define a mask for filtering
non_collinear = selector.get_support()
non_collinear[:5]


In [ ]:
# Check how many features are below the theshold
non_collinear.count(True)



In [ ]:
# Check how many features are above the theshold
non_collinear.count(False)



In [ ]:
# Make the T/F a series with the column names as the index
non_collinear_series = pd.Series(selector.get_support(), index=X_train_sm.columns)
non_collinear_series



In [ ]:
# Filter training data
training_data_to_keep = X_train_sm.values[:, non_collinear]
# Filter column names
columns = np.array(X_train_sm.columns)[non_collinear]
# Make traing data into a dataframe
X_train_no_corr = pd.DataFrame(training_data_to_keep, columns = columns)

# Filter testing data
test_data_to_keep = X_test_scaled.values[:, non_collinear]
# Make test data into a dataframe
X_test_no_corr = pd.DataFrame(test_data_to_keep, columns = columns)



In [ ]:
# Make correlation heatmap for non_collinear features
corr = no_corr_df.corr()
plt.figure(figsize=(15,15))
sns.heatmap(corr,cmap='coolwarm');



In [ ]:
# Test the performance
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train_no_corr, y_train_sm)
evaluate_classification(rf_clf, X_train_no_corr, y_train_sm, X_test_no_corr, y_test)



In [ ]:
from sklearn.feature_selection import VarianceThreshold



In [ ]:
# checking for constant-features
selector = VarianceThreshold(threshold=0.00)
selector.fit(X_train)



In [ ]:
# get support returns true/false for keeping features
keep_features = selector.get_support()
keep_features[:5]


In [ ]:
# Number of features to keep (non-constant features)
keep_features.sum()



In [ ]:
# checking for quasi-constant-features
selector = VarianceThreshold(threshold=0.01)
selector.fit(X_train)



In [ ]:
# get support returns true/false for keeping features
keep_features = selector.get_support()
keep_features[:5]



In [ ]:
# Number of features to keep (non-quasi-constant features)
keep_features.sum()



In [ ]:
# Filter based on variance
X_train_var = X_train.loc[:,keep_features]
X_test_var = X_test.loc[:,keep_features]
X_train_var



In [ ]:
# Use standardscaler to Scale
scaler = StandardScaler()
scaler.fit(X_train_var)
X_train_var_scaled = scaler.transform(X_train_var)
X_test_var_scaled = standard=scaler.transform(X_test_var)



In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 42)
X_train_var_scaled_sm, y_train_sm = smote.fit_resample(X_train_var_scaled, y_train)
y_train_sm.value_counts()



In [ ]:
# Test the performance
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train_var_scaled_sm, y_train_sm)
evaluate_classification(rf_clf, X_train_var_scaled_sm, y_train_sm, X_test_var_scaled, y_test)



# Feature Engineering 2

In [ ]:
from sklearn.feature_selection import SelectFromModel
# Instantiate a lostistic regression model
log_reg = LogisticRegression(C=1e12)
# Define the selector object using the model. Use default threshold (mean)
selector = SelectFromModel(log_reg)
# Fit the selector on the training data
selector.fit(X_train_sm ,y_train_sm)
selector



In [ ]:
# Obtain threshold used
selector.threshold_



In [ ]:
# What is the shape of the output for coefficents?
selector.estimator_.coef_.shape


In [ ]:
# View first coefficient
selector.estimator_.coef_[0,0]



In [ ]:
# Reduce this array to one dimension
flattened = selector.estimator_.coef_.flatten() 
flattened.shape



In [ ]:
# Access first value of 1d array
flattened[0]


In [ ]:
# Make the coefs a series with the column names as the index
coeffs = pd.Series(flattened, index=X_train_sm.columns)
coeffs



In [ ]:
# .get_support returns an array of T/F whether it is above threshold
above_threshold = selector.get_support()
above_threshold[0:5]



In [ ]:
# Filter the coeffs series to include only those above the threshold
coeffs[above_threshold]



In [ ]:
# .get_support returns an array of T/F whether it is above threshold
above_threshold = selector.get_support()
# Only include the features that are above the threshold in X train and X test
X_train_sel = X_train_sm.loc[:,above_threshold]
X_test_sel = X_test_scaled.loc[:,above_threshold]
X_train_sel.shape



In [ ]:
# Instantiate default random forest
rf_clf = RandomForestClassifier(random_state=42)
# Fit on selected featuers
rf_clf.fit(X_train_sel, y_train_sm)
# Evaluate with custom function
evaluate_classification(rf_clf, X_train_sel, y_train_sm, X_test_sel, y_test)



### Example Wrapper Method-Using  Forward Selection

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeClassifier



In [ ]:
# Instantiate the model
dt = DecisionTreeClassifier(random_state = 42)



In [ ]:
# Instantiate the SequentialFeatureSelector
sk_sfs = SequentialFeatureSelector(dt, n_features_to_select=50,
                                direction = 'forward', cv=2,
                                 n_jobs=-1)
sk_sfs.fit(X_train_sel,y_train_sm)



In [ ]:
# Preview array of T/F for selected (or not) features
sk_sfs.support_[0:5]



In [ ]:
# .support_ returns an array of T/F whether it is above threshold
features_to_keep = sk_sfs.support_
# Only include the features selected
X_train_50 = X_train_sel.loc[:,features_to_keep]
X_test_50 = X_test_sel.loc[:,features_to_keep]
X_train_50.shape



In [ ]:
# Instantiate default random forest
rf_clf = RandomForestClassifier(random_state=42)
# Fit on 50 selected featuers
rf_clf.fit(X_train_50, y_train_sm)
# Evaluate with custom function
evaluate_classification(rf_clf, X_train_50, y_train_sm, X_test_50, y_test)

